<a href="https://colab.research.google.com/github/joycerlz/distilbert-text-classification/blob/main/clean_essays.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspellchecker

In [ ]:
import pandas as pd
import nltk
import random
from spellchecker import SpellChecker

In [ ]:
nltk.download('wordnet')

In [ ]:
df_pre = pd.read_csv("essays.csv", encoding='latin1')

In [ ]:
df_pre.drop(columns=['#AUTHID'], inplace=True)

df_pre.rename(columns={'TEXT': 'text'}, inplace=True)
df_pre.rename(columns={'cEXT': 'ext'}, inplace=True)
df_pre.rename(columns={'cNEU': 'neu'}, inplace=True)
df_pre.rename(columns={'cAGR': 'agr'}, inplace=True)
df_pre.rename(columns={'cCON': 'con'}, inplace=True)
df_pre.rename(columns={'cOPN': 'opn'}, inplace=True)

In [ ]:
df_pre.head()

,text,ext,neu,agr,con,opn
0,"Well, right now I just woke up from a mid-day ...",n,y,y,n,y
1,"Well, here we go with the stream of consciousn...",n,n,y,n,n
2,An open keyboard and buttons to push. The thin...,n,y,n,y,y
3,I can't believe it! It's really happening! M...,y,n,y,y,n
4,"Well, here I go with the good old stream of co...",y,n,y,n,y


In [ ]:
def convert_to_labels(row):
  labels = []
  if row['agr'] == 'y':
    labels.append(0)  # Agreeableness
  if row['ext'] == 'y':
    labels.append(1)  # Extraversion
  if row['opn'] == 'y':
    labels.append(2)  # Openness
  if row['con'] == 'y':
    labels.append(3)  # Conscientiousness
  if row['neu'] == 'y':
    labels.append(4)  # Neuroticism
  return labels

def synonym_replacement(sentence, num_replacements=1):
  words = sentence.split()
  new_words = list(words)
  for _ in range(num_replacements):
    word_index = random.randint(0, len(words) - 1)
    syns = wordnet.synsets(words[word_index])
    if syns:
      synonym = syns[0].lemmas()[0].name()  # Get the first synonym
      if synonym != words[word_index]:
        new_words[word_index] = synonym
  return ' '.join(new_words)

## For each entry with one or more personality types, oversample the text section and assign all its personality types

In [ ]:
new_rows = []
spell = SpellChecker()
for index, row in df_pre.iterrows():
  labels = convert_to_labels(row)
  if labels:
    for i, label in enumerate(labels):
      if i == 0:
        new_row = row.copy()  # Use original text for the first label
      else:
        new_row = row.copy()
        new_row['text'] = synonym_replacement(row['text'], 10)  # Replace 10 words
        new_row['text'] = spell.correction(new_row['text'])

      new_row['labels'] = label
      new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
df = pd.DataFrame(new_rows)

In [ ]:
df.drop(columns=['ext'], inplace=True)
df.drop(columns=['neu'], inplace=True)
df.drop(columns=['agr'], inplace=True)
df.drop(columns=['con'], inplace=True)
df.drop(columns=['opn'], inplace=True)

In [ ]:
df.head()

,text,labels
0,"Well, right now I just woke up from a mid-day ...",0
0,"Well, right now I just woke up from a mid-day ...",2
0,"Well, right now I just woke up from a mid-day ...",4
1,"Well, here we go with the stream of consciousn...",0
2,An open keyboard and buttons to push. The thin...,2


In [ ]:
df.to_csv('essays_cleaned.csv', index=False)

In [ ]:
df['labels'].value_counts()

0    1310
1    1276
2    1271
3    1253
4    1233
Name: labels, dtype: int64

## keep only one occurence of each text and andomly assign one of its personality types

In [ ]:
new_rows = []
for index, row in df_pre.iterrows():
  labels = convert_to_labels(row)
  if labels:
    new_row = row.copy()
    new_row['labels'] = random.choice(labels)
    new_rows.append(new_row)

# Create a new DataFrame from the list of new rows
df = pd.DataFrame(new_rows)

In [ ]:
df.drop(columns=['ext'], inplace=True)
df.drop(columns=['neu'], inplace=True)
df.drop(columns=['agr'], inplace=True)
df.drop(columns=['con'], inplace=True)
df.drop(columns=['opn'], inplace=True)

In [ ]:
df.head()

,text,labels
0,"Well, right now I just woke up from a mid-day ...",0
1,"Well, here we go with the stream of consciousn...",0
2,An open keyboard and buttons to push. The thin...,4
3,I can't believe it! It's really happening! M...,0
4,"Well, here I go with the good old stream of co...",2


In [ ]:
df.to_csv('essays_cleaned_v2.csv', index=False)

In [ ]:
df['labels'].value_counts()

4    539
0    488
2    476
3    453
1    451
Name: labels, dtype: int64